# Capacitated Facility Location Problem
Consider the Capacitated Facility Location Problem introduced in Chapter 7 of the Benders Decomposition lecture notes. An instance of the problem is provided below together with a class for the problem and a class for the full model.
- Q1 Solve the instance of the problem using Benders decomposition
- Q2 Compare its solution to that of the non-decomposed problem to ensure it is correct.

## Data

In [1]:
import random as r
n_locations = 15
n_customers = 30

r.seed(1)
# Random fixed costs between 100 and 300
fixed_costs = [(100 + (r.random() * 200)) for i in range(n_locations)]

# Random delivery costs between 10 and 40
delivery_costs = {(i,j):(10 + (r.random() * 30)) for j in range(n_customers) for i in range(n_locations)}

# Random demands between 50 and 100
demands = [(50 + (r.random() * 50)) for j in range(n_customers)]

# Random capacities between 100 and 140
capacities = [(150 + (r.random() * 50)) for i in range(n_locations)]


## Class for the Facility Location Problem.

In [2]:
class FacilityLocationProblem:

    def __init__(self,n_facilities,n_customers,fixed_costs,delivery_costs,demands,capacity):
        self.n_facilities = n_facilities
        self.n_customers = n_customers
        self.fixed_costs = fixed_costs
        self.delivery_costs = delivery_costs
        self.demands = demands
        self.capacity = capacity

We can now create an instance of the Facility Location Problem

In [3]:
flp = FacilityLocationProblem(n_locations, n_customers, fixed_costs, delivery_costs, demands, capacities)

## Class for the full model

In [4]:
class FullModel:

    def __init__(self, flp:FacilityLocationProblem):
        self.flp = flp
        self.m = Model()

        # Creates the variables
        self.y = self.m.addVars(flp.n_facilities, flp.n_customers, name="y")
        self.x = self.m.addVars(flp.n_facilities, vtype=GRB.BINARY, name="x")

        # Creates the objective
        self.m.setObjective(self.x.prod(flp.fixed_costs) + self.y.prod(flp.delivery_costs), GRB.MINIMIZE)

        # Constraints
        
        self.m.addConstrs(self.y.sum(i, '*') <= flp.capacity[i] * self.x[i] for i in range(flp.n_facilities))
        self.m.addConstrs(self.y.sum('*', j) >= flp.demands[j] for j in range(flp.n_customers))

    def solve(self):
        self.m.optimize()

    def print_solution(self):
        for i in range(self.flp.n_facilities):
            print('%s %g' % (self.x[i].varName, self.x[i].x))
        print('Obj: %g' % self.m.objVal)
